In [1]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
%matplotlib inline

# Data Formatting

```python
train = pd.read_csv('../data/train.csv')
train.shape
weather = pd.read_csv('../data/weather.csv')
weather.shape

weather['Timestamp'] = pd.to_datetime(weather['Timestamp'])
train['Timestamp'] = pd.to_datetime(train['Timestamp'])

train_weather = pd.merge(weather, train, how = 'right', on = ['SiteId', 'Timestamp'])

train_weather['Minutes'] = [time.minute for time in train_weather['Timestamp']]
train_weather['Hour'] = [time.hour for time in train_weather['Timestamp']]
train_weather['day'] = [time.day for time in train_weather['Timestamp']]
train_weather['wday'] = [time.weekday() for time in train_weather['Timestamp']]
train_weather['month'] = [time.month for time in train_weather['Timestamp']]
train_weather['Year'] = [time.year for time in train_weather['Timestamp']]

train_weather.head()

test = pd.read_csv('../data/submission_format.csv')
test_weather = pd.merge(weather, test, how = 'right', on = ['SiteId', 'Timestamp'])
test_weather['Timestamp'] = pd.to_datetime(test_weather['Timestamp'])

test_weather['Minutes'] = [time.minute for time in test_weather['Timestamp']]
test_weather['Hour'] = [time.hour for time in test_weather['Timestamp']]
test_weather['day'] = [time.day for time in test_weather['Timestamp']]
test_weather['wday'] = [time.weekday() for time in test_weather['Timestamp']]
test_weather['month'] = [time.month for time in test_weather['Timestamp']]
test_weather['Year'] = [time.year for time in test_weather['Timestamp']]

test_weather.head()
```

In [9]:
train_weather['wday'] = [time.weekday() for time in train_weather['Timestamp']]
test_weather['wday'] = [time.weekday() for time in test_weather['Timestamp']]

In [10]:
train_weather.to_csv('../data/train_weather.csv')
test_weather.to_csv('../data/test_weather.csv')

## Read in Formatted Data

In [3]:
train_weather = pd.read_csv('../data/train_weather.csv')
test_weather = pd.read_csv('../data/test_weather.csv')

train_weather['Timestamp'] = pd.to_datetime(train_weather['Timestamp'])
test_weather['Timestamp'] = pd.to_datetime(test_weather['Timestamp'])

## Time Difference in Hours Column

In [14]:
train_weather['time_diff'] = train_weather['Timestamp'].diff()
train_weather['time_diff'] = [(time.total_seconds()/3600) for time in train_weather['time_diff']]

## Find averages for all buildings

In [70]:
averages = train_weather.groupby(['SiteId', 'time_diff']).mean()

In [71]:
averages = averages.reset_index()

In [72]:
averages_quarter = averages[averages['time_diff'] == 0.25]
averages_hour = averages[averages['time_diff'] == 1.0]
averages_day = averages[averages['time_diff'] == 24.0]

In [73]:
averages_quarter = averages_quarter[['SiteId', 'time_diff', 'Value']].rename(columns={'Value': 'avg_quarter'})
averages_hour = averages_hour[['SiteId', 'time_diff', 'Value']].rename(columns={'Value': 'avg_hour'})
averages_day = averages_day[['SiteId', 'time_diff', 'Value']].rename(columns={'Value': 'avg_day'})

In [74]:
averages = pd.merge(averages_quarter, averages_hour, how = 'outer', on = 'SiteId')
averages = pd.merge(averages, averages_day, how = 'outer', on = 'SiteId')

In [75]:
averages.head()

SiteId  time_diff_x   avg_quarter  time_diff_y      avg_hour  time_diff  \
0       5         0.25  25595.227516          1.0  26001.779204        NaN   
1       6         0.25  34005.855622          1.0  34487.195803        NaN   
2       8         0.25  47201.535197          1.0  48409.949416        NaN   
3       9         0.25   3357.436132          1.0   3353.627457        NaN   
4      11         0.25   2359.358924          NaN           NaN        NaN   

   avg_day  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN

In [76]:
averages = averages[['SiteId', 'avg_quarter', 'avg_hour', 'avg_day']]

In [77]:
averages.to_csv('../data/averages.csv')

In [78]:
meta = pd.read_csv('../data/metadata.csv')
meta.head()

SiteId       Surface  Sampling  BaseTemperature  MondayIsDayOff  \
0       1   1387.205119      15.0             18.0           False   
1       2   6098.278376      30.0             18.0           False   
2       3  10556.293605       5.0             18.0           False   
3       5  12541.181277      30.0             18.0           False   
4       6   9150.195373      30.0             18.0           False   

   TuesdayIsDayOff  WednesdayIsDayOff  ThursdayIsDayOff  FridayIsDayOff  \
0            False              False             False           False   
1            False              False             False           False   
2            False              False             False           False   
3            False              False             False           False   
4            False              False             False           False   

   SaturdayIsDayOff  SundayIsDayOff  
0              True            True  
1              True            True  
2              True           False  
3              True            True  
4              True            True

In [79]:
meta_new = pd.merge(meta, averages, how = 'outer', on = 'SiteId')

In [81]:
meta_new.head()

SiteId       Surface  Sampling  BaseTemperature  MondayIsDayOff  \
0       1   1387.205119      15.0             18.0           False   
1       2   6098.278376      30.0             18.0           False   
2       3  10556.293605       5.0             18.0           False   
3       5  12541.181277      30.0             18.0           False   
4       6   9150.195373      30.0             18.0           False   

   TuesdayIsDayOff  WednesdayIsDayOff  ThursdayIsDayOff  FridayIsDayOff  \
0            False              False             False           False   
1            False              False             False           False   
2            False              False             False           False   
3            False              False             False           False   
4            False              False             False           False   

   SaturdayIsDayOff  SundayIsDayOff   avg_quarter      avg_hour       avg_day  
0              True            True           NaN           NaN  4.421744e+06  
1              True            True           NaN  67524.364653           NaN  
2              True           False           NaN           NaN  6.591267e+06  
3              True            True  25595.227516  26001.779204           NaN  
4              True            True  34005.855622  34487.195803           NaN

In [103]:
patterns = []
for i in range(len(meta_new)):
    row = meta_new.iloc[i]
    weekly = list(row[4:11])
    if weekly == [0, 0, 0, 0, 0, 1, 1]:
        patterns.append('5-2')
    elif weekly == [0, 0, 0, 0, 0, 1, 0]:
        patterns.append('Sat-off')
    elif weekly == [0, 0, 0, 0, 0, 0, 1]:
        patterns.append('Sun-off')
    elif weekly == [0, 0, 0, 0, 0, 0, 0]:
        patterns.append('All-on')
    elif weekly == [0, 0, 0, 0, 1, 1, 1]:
        patterns.append('4-3')
    else:
        patterns.append('NA')

In [104]:
from collections import Counter
counter = Counter()

In [105]:
Counter(patterns)

Counter({'4-3': 1, '5-2': 252, 'All-on': 8, 'Sat-off': 1, 'Sun-off': 5})

In [106]:
meta_new['pattern'] = patterns

In [107]:
meta_new.head()

SiteId       Surface  Sampling  BaseTemperature  MondayIsDayOff  \
0       1   1387.205119      15.0             18.0           False   
1       2   6098.278376      30.0             18.0           False   
2       3  10556.293605       5.0             18.0           False   
3       5  12541.181277      30.0             18.0           False   
4       6   9150.195373      30.0             18.0           False   

   TuesdayIsDayOff  WednesdayIsDayOff  ThursdayIsDayOff  FridayIsDayOff  \
0            False              False             False           False   
1            False              False             False           False   
2            False              False             False           False   
3            False              False             False           False   
4            False              False             False           False   

   SaturdayIsDayOff  SundayIsDayOff   avg_quarter      avg_hour       avg_day  \
0              True            True           NaN           NaN  4.421744e+06   
1              True            True           NaN  67524.364653           NaN   
2              True           False           NaN           NaN  6.591267e+06   
3              True            True  25595.227516  26001.779204           NaN   
4              True            True  34005.855622  34487.195803           NaN   

   pattern  
0      5-2  
1      5-2  
2  Sat-off  
3      5-2  
4      5-2

In [108]:
meta_new = meta_new[['SiteId', 'Surface', 'avg_quarter', 'avg_hour', 'avg_day', 'pattern']]

In [110]:
meta_new.to_csv('../data/meta_augmented.csv')

# Work Day

## Training Data

In [12]:
id_list = set(train_weather['SiteId'])
wday_list = set(train_weather['wday'])

In [ ]:
i = 0 
for site in id_list:
    site_row = (meta[meta['SiteId'] == site])
    print(site)
    for wd in wday_list:
        train_weather.ix[(train_weather['SiteId'] == site) & (train_weather['wday'] == wd), 'work'] = site_row.iloc[0, 4 + wd]

1
2
3
5
6
7


In [ ]:
train_weather.head()

In [124]:
work_days = []

for i in train_weather.index:
    if i % 100000 == 0:
        print('Another 100,000', i)
    row = train_weather.iloc[i]
    site = row['SiteId']
    wday = row['wday']
    day_status = meta[meta['SiteId'] == site].iloc[0, 4 + wday]
    if day_status:
        work_days.append(0)
    else: 
        work_days.append(1)

Another 100,000 0
Another 100,000 100000
Another 100,000 200000
Another 100,000 300000
Another 100,000 400000
Another 100,000 500000
Another 100,000 600000
Another 100,000 700000
Another 100,000 800000


KeyboardInterrupt: 

In [ ]:
train_weather['work'] = work_days

In [ ]:
train_weather.head(20)

## Testing Data

In [ ]:
test_work_days = []

for i in test_weather.index:
    row = test_weather.iloc[i]
    site = row['SiteId']
    wday = row['Timestamp'].weekday()
    day_status = meta[meta['SiteId'] == site].iloc[0, 4 + wday]
    if day_status:
        test_work_days.append(0)
    else: 
        test_work_days.append(1)

In [ ]:
test_weather['work'] = test_work_days

# Add Holidays

In [ ]:
holidays = pd.read_csv('../data/holidays.csv')
holidays['Date'] = pd.to_datetime(holidays['Date'])
holidays.head()

## Training Data

In [ ]:
holiday_days = []

for i in train_weather.index:
    row = train_weather.iloc[i]
    site = row['SiteId']
    date = row['Timestamp'].date()
    
    status = 0
    for ii in holidays.index:
        holiday_row = holidays.iloc[i]
        if (holiday_row['Date'].date() == date) & 
        (holiday_row['SiteId'] == site):
            status = 1
        
    holiday_days.append(status)
    
    if date in holidays['Date']

# Fill in Missing Values? 

In [ ]:
from sklearn.preprocessing import Imputer

impute = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)

In [ ]:
df = train_weather[train_weather['SiteId'] == 5].sort_values(['Timestamp', 'Distance'])
df = df.drop_duplicates('Timestamp', keep = 'first')

In [ ]:
meta.head()

In [ ]:
sub = pd.read_csv('../data/submission_frequency.csv')
sub['hours'] = sub['ForecastPeriodNS'] / (3600 * 1e9)

In [ ]:
sub.head(20)

## Sort first by timestamp and then by weather station distance

In [ ]:
site_two = train_weather[train_weather['SiteId'] == 2].sort_values(['Timestamp', 'Distance'])

In [ ]:
site_two.head()

## Fill NA temperature values

In [ ]:
site_two['Temperature'] = site_two['Temperature'].fillna(-99)

## Drop Duplicates

In [ ]:
site_two.drop_duplicates('Timestamp', keep='first', inplace=True)

In [ ]:
site_two.head()

## Fill NA in value column

In [ ]:
site_two['Value'] = site_two['Value'].fillna(method='ffill')

## Timestamp to Float for Now

In [ ]:
site_two['Timestamp'] = [time.days * 3600 * 24 for time in (site_two['Timestamp'] - min(site_two['Timestamp']))]

# Drop Unused columns

In [ ]:
site_two = site_two.drop(columns=['Unnamed: 0', 'Distance', 'SiteId', 'ForecastId', 'Value'])

# Processing Function

In [ ]:
def process_site(site_id, data='train'):
    
    if data == 'train':
        df = train_weather[train_weather['SiteId'] == site_id].sort_values(['Timestamp', 'Distance'])
        
    else:
        df = test_weather[test_weather['SiteId'] == site_id].sort_values(['Timestamp', 'Distance'])
    
    min_date = min(train_weather[train_weather['SiteId'] == site_id]['Timestamp'])
    
    df['Temperature'] = df['Temperature'].fillna(-99)
    
    df = df.drop_duplicates('Timestamp', keep = 'first')
    
    labels = df['Value'].fillna(method='ffill')
    labels = labels.fillna(0)
    
    df = df.drop(columns = ['Unnamed: 0', 'Distance', 'SiteId', 'ForecastId', 'Value'])
    
    df['Timestamp'] = [(time.days*3600*24) for time in (df['Timestamp'] - min_date)] 
    df = df.fillna(-99)
    
    return df, labels

In [ ]:
site_one_df, labels = process_site(1)
site_one_df.head()

# Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin_model = LinearRegression()
lin_model.fit(site_one_df, labels)

# Submission Data

In [ ]:
test_weather_df, test_labels = process_site(1, data='test')

In [ ]:
test_weather_df.head()

## Make Predictions

In [ ]:
from itertools import chain
from IPython.display import clear_output

In [ ]:
predictions = lin_model.predict(test_weather_df)

In [ ]:
len(predictions)

In [ ]:
import sys
import time

In [ ]:
predictions = []

for id in set(test['SiteId']):
    print(id)
    
    lin_model = LinearRegression()
    train_x, train_y = process_site(id, data = 'train')
    test_x, test_y = process_site(id, data = 'test')
    
    lin_model.fit(train_x, train_y)
    predicted = lin_model.predict(test_x)
    predictions.append(predicted)
    
    clear_output()

In [ ]:
test_submit = pd.read_csv('../data/submission_format.csv')

In [ ]:
predictions_flat = list(chain(*predictions))

# Random Forest

In [ ]:
from IPython.display import clear_output
import sys

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_predictions = []

for i, id in enumerate(set(test['SiteId'])):

    print('Percentage Complete: {:.2f}'.format(100 * i / len(set(test['SiteId']))))
    tree_reg = RandomForestRegressor(n_estimators=500)
    train_x, train_y = process_site(id, data = 'train')
    test_x, test_y = process_site(id, data = 'test')
    
    tree_reg.fit(train_x, train_y)
    predicted = lin_model.predict(test_x)
    clear_output()
    rf_predictions.append(predicted)

In [ ]:
rf_predictions = list(chain(*predictions))
test_submit = pd.read_csv('../data/submission_format.csv')
test_submit['Value'] = rf_predictions

In [ ]:
test_submit.to_csv('../submissions/second_rf.csv', index=False)

# Feature Engineering